In [1]:
import commonsetting
from models import perceptual_network, Encoder, Class_out, Conf_out
from dataloader import CustomImageDataset, concatenate_transform_steps
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch import nn
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd


if __name__ == "__main__":
    pretrained_model_name   = commonsetting.pretrained_model_name
    hidden_layer_size       = commonsetting.hidden_layer_size
    hidden_activation_name  = commonsetting.hidden_activation_name
    hidden_activation       = commonsetting.hidden_activation
    hidden_dropout          = commonsetting.hidden_dropout
    resize                  = commonsetting.image_resize
    in_shape                = (1,3,resize,resize)

    SimpleCNN_args = dict(
        pretrained_model_name   = pretrained_model_name,
        hidden_layer_size       = hidden_layer_size,
        hidden_activation       = hidden_activation,
        hidden_dropout          = hidden_dropout,
        hidden_layer_type       = commonsetting.hidden_layer_type,
        output_layer_size       = commonsetting.output_layer_size,
        in_shape                = (1,3,resize,resize),
        retrain_encoder         = commonsetting.retrain_encoder,
        device                  = commonsetting.device,
        )
        

    tranformer_steps = concatenate_transform_steps(image_resize=commonsetting.image_resize, rotate=0)

    dataset_test = CustomImageDataset(commonsetting.test_dir,label_map=commonsetting.label_map , transform=tranformer_steps)
    dataloader_test = DataLoader(dataset_test, batch_size=commonsetting.batch_size, shuffle=True, num_workers=commonsetting.num_workers)
    SimpleCNN = perceptual_network(**SimpleCNN_args)
    SimpleCNN.load_state_dict(torch.load("../models/train_pixel_0.6/simplecnn_bs32e4i224h300.h5"))
    for p in SimpleCNN.parameters(): p.requires_grad = False
    # define loss function
    classification_loss = nn.BCELoss()
    # when there is no noise
    SimpleCNN.eval()
    SimpleCNN.to(commonsetting.device)
    with torch.no_grad():
        # extract features of each image
        y_true = []
        y_pred = []
        y_correct_pred = []
        confidence_out = []
        for idx_batch, (batch_image, batch_label) in tqdm(enumerate(dataloader_test)):
            batch_label = torch.vstack(batch_label).T.float()
            #记得每一次处理数据之前要做这一步
            batch_image = batch_image.to(commonsetting.device)
            batch_label = batch_label.to(commonsetting.device)
            
            features,hidden_representation,prediction, confidence =  SimpleCNN(batch_image.to(commonsetting.device))
            correct_preds = batch_label.clone().detach().argmax(1)==prediction.clone().detach().argmax(1)
            correct_preds = correct_preds.float()

            correct_preds = torch.vstack([1-correct_preds, correct_preds]).T.float()
            
            
            y_correct_pred.append(correct_preds.detach().cpu().numpy())
            y_true.append(batch_label.detach().cpu().numpy())
            y_pred.append(prediction.detach().cpu().numpy())
            confidence_out.append(confidence.detach().cpu().numpy())
    
    y_pred = np.concatenate(y_pred, axis=0)
    y_true = np.concatenate(y_true, axis=0)
    y_correct_pred = np.concatenate(y_correct_pred, axis=0)
    confidence_out = np.concatenate(confidence_out, axis=0)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [2]:
from sklearn.metrics import roc_auc_score, accuracy_score

In [ ]:
roc_auc_score(y_true, y_pred)

0.7939467592592594

In [ ]:
roc_auc_score(y_correct_pred, confidence_out)

In [ ]:
y_pred.shape

In [ ]:
from collections import Counter
Counter(y_pred.argmax(1))

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.hist(confidence_out[:,0])

In [ ]:
confidence_out.argmax(1)

In [ ]:
confidence_onehot = np.vstack([1-confidence_out.argmax(1), confidence_out.argmax(1)]).T

In [ ]:
from collections import Counter
Counter(y_correct_pred[:,1])

In [ ]:
def type2_correct(confidence, correct_trial):
    confidence_onehot = np.vstack([1-confidence.argmax(1), confidence.argmax(1)]).T
    idx_correct_trial = correct_trial == 1#正确的trial
    total_correct = sum(idx_correct_trial.astype(int))
    type2_acc =confidence_onehot[:, 1][idx_correct_trial]
    return  sum(type2_acc), total_correct

In [ ]:
type2_correct, total_correct = type2_correct(confidence_out, y_correct_pred[:,1])

In [ ]:
type2_correct/total_correct

In [ ]:
type2_correct

In [ ]:
total_correct

In [ ]:
y_true

In [ ]:
response = y_pred.argmax(1) == 1
response.astype(int)

In [ ]:
y_pred

In [ ]:
rating = confidence_out.argmax(1) == 1

In [ ]:
df_cnn = pd.DataFrame({"response":response.tolist(), 
              "stim":y_true[:, 1].astype(int).tolist(), 
             "acc":y_correct_pred[:, 1].astype(int).tolist(), 
             "confidence":rating.astype(int).tolist()})

In [ ]:
df_cnn.to_csv("../result/reuslt_cnn.csv")